In [3]:
import os, sagemaker, boto3
from datetime import datetime
from sagemaker import s3
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel
from sagemaker.deserializers import JSONDeserializer

In [18]:
s3_client = boto3.client('s3')
response = s3_client.list_buckets()

print(response)
for bucket in response['Buckets']:
    if 'model' in bucket["Name"]:
        bucket = 's3://' + bucket["Name"]
        break

print(f'Bucket: {bucket}')
sess = sagemaker.Session(default_bucket=bucket.split('s3://')[-1])

prefix = "yolov8/parking-custom-endpoint"

{'ResponseMetadata': {'RequestId': 'RSD3WVJ8BRSV4AW7', 'HostId': '/om1x1E9RX9wRhXAM530d1DmYqjGCamIKa/AlokUMCrzpDejjY/U+i9zc9ev1hGp92aYP2sObTk=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '/om1x1E9RX9wRhXAM530d1DmYqjGCamIKa/AlokUMCrzpDejjY/U+i9zc9ev1hGp92aYP2sObTk=', 'x-amz-request-id': 'RSD3WVJ8BRSV4AW7', 'date': 'Sun, 03 Dec 2023 23:50:28 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'models-ml-cc', 'CreationDate': datetime.datetime(2023, 12, 3, 1, 22, tzinfo=tzutc())}], 'Owner': {'DisplayName': 'gvemula', 'ID': 'b45674900bd6b61840674592aee8fd770ce2e6a1fa1cb74a09f8859c25e1e3b8'}}
Bucket: s3://models-ml-cc


In [19]:
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

role = "AmazonSageMaker-ExecutionRole-20231202T200293"
print(f'Role: {role}')


model_data = s3.S3Uploader.upload("/Users/gouravvemula/Downloads/parking-model/model.tar.gz", bucket + "/" + prefix)
print(f'Model Data: {model_data}')

Role: AmazonSageMaker-ExecutionRole-20231202T200293
Model Data: s3://models-ml-cc/yolov8/parking-custom-endpoint/model.tar.gz


In [20]:
model = PyTorchModel(entry_point='inference.py',
                     model_data=model_data, 
                     framework_version='1.12', 
                     py_version='py38',
                     role=role,
                     env={'TS_MAX_RESPONSE_SIZE':'20000000', 'YOLOV8_MODEL': 'yolov8m.pt'},
                     sagemaker_session=sess)

In [21]:
INSTANCE_TYPE = 'ml.m4.xlarge'
ENDPOINT_NAME = 'yolov8-pytorch-' + str(datetime.utcnow().strftime('%Y-%m-%d-%H-%M-%S-%f'))

# Store the endpoint name in the history to be accessed by 2_TestEndpoint.ipynb notebook
%store ENDPOINT_NAME
print(f'Endpoint Name: {ENDPOINT_NAME}')

predictor = model.deploy(initial_instance_count=1, 
                         instance_type=INSTANCE_TYPE,
                         deserializer=JSONDeserializer(),
                         endpoint_name=ENDPOINT_NAME)

Stored 'ENDPOINT_NAME' (str)
Endpoint Name: yolov8-pytorch-2023-12-03-23-50-32-390066
--------!